In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

# Inputs

In [2]:
'''
print("Give me some informations on this film : ")
title = str(input("Title :"))
year = input("Year :")
parentalAdvisor = str(input("Parental advisor :"))
duree = input("Film's duration in minutes :")
resume = str(input("Resume :"))
genre = str(input("Genre (for multi-genres separate by commas) :"))
'''
title = 'Miss Jerry'
year = 1894
parentalAdvisor = 'Not Rated'
duree = 70.0
resume = 'Story of Ned Kelly, an infamous 19th-century Australian outlaw.'
genre = 'Biography'

my_film = pd.DataFrame(columns=['title','year','parentalAdvisor','duree','resume','genre'],
                          data=[[ title , year , parentalAdvisor , duree , resume , genre ]])

target_name = 'imdbRating'

In [3]:
my_film["genres"] = my_film['genre']

# Convert string Genre to list
my_film["genres"] = [ str(x).replace(' ', '') for x in my_film["genres"][:] ] 
my_film["genres"] = [ x.split(",") for x in my_film["genres"][:] ]

# https://www.sbert.net/docs/quickstart.html#comparing-sentence-similarities
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')
# from unittest import result

my_film['genre'] = ''
my_film['themaScore'] = 0.0

for i in range( len(my_film) ):
    resume = my_film.iloc[i]['resume']
    genres = my_film.iloc[i]['genres']
    bestScore = 0.0
    bestGenre = ''
    for genre in genres:
        emb1 = model.encode(str(genre))
        emb2 = model.encode(str(resume))
        cos_sim = util.cos_sim(emb1, emb2)
        result = cos_sim.tolist()[0][0]
        if result > bestScore:
            bestScore = result
            bestGenre = genre
    # print(i, '/', len(dataset))
    my_film.themaScore[i] = bestScore
    my_film.genre[i] = bestGenre

my_film = my_film.drop(columns=['genres','title','resume'])

my_film = my_film.reindex(columns = [col for col in my_film.columns if col != target_name] + [target_name])

<ipython-input-3-324a7ddea079>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_film.themaScore[i] = bestScore
<ipython-input-3-324a7ddea079>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_film.genre[i] = bestGenre


## Original dataset 

In [4]:
dataset = pd.read_csv('src/dataset.csv', delimiter=',', on_bad_lines='skip')
dataset = dataset.drop(columns=['title', 'director', 'resume'])
my_film = my_film[dataset.columns.to_list()]
print(dataset.head(1))
print(my_film.head(1))

   year  imdbRating parentalAdvisor  duree    genre  themaScore
0  1894         5.3             NaN   45.0  Romance    0.151441
   year  imdbRating parentalAdvisor  duree      genre  themaScore
0  1894         NaN       Not Rated   70.0  Biography    0.304189


## Separation Varibles / Target

In [5]:
X = dataset.drop(columns= [target_name])
X_train = dataset.drop(columns= [target_name])
Y_train = dataset[:][target_name]

X_test = my_film.drop(columns= [target_name])
Y_test = my_film[:][target_name]

## Pipeline Préprocessor 

In [6]:
## Médiane quantitative & plus fréquent qualitatif

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import( OneHotEncoder, StandardScaler, LabelEncoder )
from sklearn.compose import ColumnTransformer

# Create pipeline for numeric features
numeric_features = X.select_dtypes([np.number]).columns # Automatically detect positions of numeric columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # missing values will be replaced by columns' median
    ('scaler', StandardScaler())
])

# Create pipeline for categorical features
categorical_features = X.select_dtypes("object").columns # Automatically detect positions of categorical columns
categorical_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # missing values will be replaced by most frequent value
    ('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])

# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X_train = preprocessor.fit_transform(X_train) # Preprocessing influenceur
X_test = preprocessor.transform(X_test) # Preprocessing copieur


## Training
### Random Forest

In [7]:
from sklearn.ensemble import RandomForestRegressor
import joblib

model = joblib.load("src/random_forest.joblib")

my_imdb_score = model.predict(X_test)[0]

my_imdb_score

5.79